In [1]:
import os, sys
import glob

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

import datetime
import time

# df_db와 df_inout 연결

In [2]:
df_db = pd.read_csv(f"./final_db/df_db_hday_rvs.csv", index_col=[0])
df_inout = pd.read_csv("./final_db/2015_subway_day_hour_1234.csv", index_col=[0])

/Users/bowoojang/Desktop/idk/venv/univ_env/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [10]:
df_inout.columns = ['date', 'line', 'stt', 'inout', ] + list(df_inout.columns)[-20:]
df_inout['line'] = df_inout['line'].apply(lambda x: f"{x}호선")
df_inout


,date,line,stt,inout,5,6,7,8,9,10,...,15,16,17,18,19,20,21,22,23,24
0,2015-01-01,1호선,서울역,승차,441.0,399.0,572.0,1106.0,1474.0,2020.0,...,4036.0,4030.0,3666.0,3738.0,3091.0,2749.0,3247.0,2437.0,1279.0,73.0
1,2015-01-01,1호선,시청,승차,101.0,95.0,104.0,127.0,233.0,308.0,...,1038.0,1027.0,1176.0,1133.0,850.0,950.0,612.0,497.0,192.0,6.0
2,2015-01-01,1호선,종각,승차,898.0,482.0,281.0,291.0,328.0,449.0,...,1530.0,2012.0,2176.0,2047.0,1767.0,1834.0,1825.0,1666.0,736.0,37.0
3,2015-01-01,1호선,종로3가,승차,371.0,212.0,155.0,189.0,347.0,491.0,...,1537.0,1791.0,1954.0,1526.0,1297.0,1149.0,1257.0,953.0,415.0,15.0
4,2015-01-01,1호선,종로5가,승차,57.0,101.0,83.0,128.0,209.0,323.0,...,915.0,984.0,948.0,962.0,722.0,576.0,431.0,241.0,102.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82485,2015-12-27,3호선,충무로,하차,26.0,45.5,59.0,91.5,132.5,215.5,...,418.0,912.5,923.0,671.5,428.5,288.0,362.5,211.5,65.0,0.0
82486,2015-12-28,3호선,충무로,하차,29.0,82.0,161.0,231.0,241.5,251.0,...,567.5,656.5,938.5,2220.5,1429.0,842.0,1127.5,751.0,271.0,41.5
82487,2015-12-29,3호선,충무로,하차,33.5,83.0,163.0,249.0,227.5,271.0,...,601.5,664.0,1082.0,2208.5,1316.0,775.0,828.5,612.5,313.0,47.5
82488,2015-12-30,3호선,충무로,하차,35.0,77.5,158.5,250.5,223.5,291.5,...,608.0,759.5,1261.5,2387.5,1378.5,768.5,1158.5,736.5,364.0,74.0


df_inout만 연결해보자. : 1호선 작동하고 쥬겄어용....

In [11]:
df_db = pd.read_csv(f"./final_db/df_db_hday_rvs.csv", index_col=[0])
df_db = df_db.drop_duplicates()

df_db1 = pd.read_csv(f"./final_db/df_db_hday_rvs_inout_line_1.csv", index_col = 0)
df_db1 = df_db1.drop_duplicates()
df_db1 = df_db1.reset_index(drop=True)

df_db.loc[df_db['line'] == '1호선', ['in', 'out']] = df_db1.loc[df_db1['line'] == '1호선', ['in', 'out']].copy()

# df_db2 = pd.read_csv(f"./final_db/df_db_hday_rvs_inout_line_2.csv", index_col = 0)
# df_db2 = df_db2.drop_duplicates()
# df_db2 = df_db2.reset_index(drop=True)

# df_db.loc[df_db['line'] == '2호선', ['in', 'out']] = df_db2.loc[df_db2['line'] == '2호선', ['in', 'out']].copy()


/Users/bowoojang/Desktop/idk/venv/univ_env/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


3, 4호선은 왜..? 

In [12]:
st_t = time.time()
for line in df_db['line'].unique():
    print(line)
    
    if line in ['1호선', ]:#'2호선']:
        continue

    cond_db1 = df_db['line']==line
    cond_inout1 = df_inout['line']==line
    
    for date in df_db['date'].unique():
        cond_db2 = df_db['date']==date
        cond_inout2 = df_inout['date']==date
        
        for stt in df_db.loc[cond_db1, 'from_stt'].unique():
            cond_db3 = df_db['from_stt']== stt
            cond_inout3 = df_inout['stt']== stt
            
            cond_in = df_inout['inout'] == '승차'
            cond_out = df_inout['inout'] == '하차'

            copy_in = df_inout.loc[cond_inout1&cond_inout2&cond_inout3&cond_in, '5':].to_numpy().reshape(-1,1)
            copy_out = df_inout.loc[cond_inout1&cond_inout2&cond_inout3&cond_out, '5':].to_numpy().reshape(-1,1)
            
            try:
                df_db.loc[cond_db1&cond_db2&cond_db3, 'in'] = copy_in
                df_db.loc[cond_db1&cond_db2&cond_db3, 'out'] = copy_out
            except:
                try:
                    copy_in = np.append(copy_in, copy_in, axis=0)
                    copy_out = np.append(copy_out, copy_out, axis=0)

                    df_db.loc[cond_db1&cond_db2&cond_db3, 'in'] = copy_in
                    df_db.loc[cond_db1&cond_db2&cond_db3, 'out'] = copy_out
                except:
                    print(line, date, stt)
                    df_err = df_db.loc[cond_db1&cond_db2&cond_db3, :].copy()
                    print("$"*30)
                    print(df_inout.loc[cond_inout1&cond_inout2&cond_inout3&cond_in, '5':])
                    break
                
        #break
        print(f"Date {date} end: {time.time() - st_t}")
    #break
    print(f"Line no.{line} end: {time.time() - st_t}")

1호선
2호선
Date 2015-01-01 end: 6.5435028076171875
Date 2015-01-02 end: 13.013556957244873
Date 2015-01-03 end: 19.252059936523438
Date 2015-01-04 end: 25.662090063095093
Date 2015-01-05 end: 32.11825489997864
Date 2015-01-06 end: 38.43532872200012
Date 2015-01-07 end: 44.88328289985657
Date 2015-01-08 end: 51.15684795379639
Date 2015-01-09 end: 57.3810179233551
Date 2015-01-10 end: 63.6329460144043
Date 2015-01-11 end: 70.11491584777832
Date 2015-01-12 end: 76.43938279151917
Date 2015-01-13 end: 82.93486499786377
Date 2015-01-14 end: 89.2602059841156
Date 2015-01-15 end: 95.38522791862488
Date 2015-01-16 end: 101.59331178665161
Date 2015-01-17 end: 107.94352078437805
Date 2015-01-18 end: 114.13457489013672
Date 2015-01-19 end: 120.50332188606262
Date 2015-01-20 end: 126.90942811965942
Date 2015-01-21 end: 133.20261192321777
Date 2015-01-22 end: 139.4665560722351
Date 2015-01-23 end: 145.78431701660156
Date 2015-01-24 end: 152.14642190933228
Date 2015-01-25 end: 158.54977178573608
Date 20

Date 2015-07-29 end: 1325.0138709545135
Date 2015-07-30 end: 1331.3355250358582
Date 2015-07-31 end: 1337.506206035614
Date 2015-08-01 end: 1343.6087980270386
Date 2015-08-02 end: 1349.7440009117126
Date 2015-08-03 end: 1355.839341878891
Date 2015-08-04 end: 1361.942568063736
Date 2015-08-05 end: 1368.0234508514404
Date 2015-08-06 end: 1374.1384499073029
Date 2015-08-07 end: 1380.2435269355774
Date 2015-08-08 end: 1386.3217899799347
Date 2015-08-09 end: 1392.5438528060913
Date 2015-08-10 end: 1399.0399949550629
Date 2015-08-11 end: 1405.4141347408295
Date 2015-08-12 end: 1411.7819547653198
Date 2015-08-13 end: 1418.01278591156
Date 2015-08-14 end: 1424.238306760788
Date 2015-08-15 end: 1430.471254825592
Date 2015-08-16 end: 1436.909828901291
Date 2015-08-17 end: 1443.2190370559692
Date 2015-08-18 end: 1449.5978441238403
Date 2015-08-19 end: 1455.8763160705566
Date 2015-08-20 end: 1462.2278759479523
Date 2015-08-21 end: 1468.663822889328
Date 2015-08-22 end: 1474.9753289222717
Date 2015

Date 2015-02-20 end: 2560.6847817897797
Date 2015-02-21 end: 2565.385099887848
Date 2015-02-22 end: 2570.1585738658905
Date 2015-02-23 end: 2574.893807888031
Date 2015-02-24 end: 2579.64777302742
Date 2015-02-25 end: 2584.4470160007477
Date 2015-02-26 end: 2589.294275999069
Date 2015-02-27 end: 2594.0234818458557
Date 2015-02-28 end: 2598.8870120048523
Date 2015-03-01 end: 2603.759845018387
Date 2015-03-02 end: 2608.690685033798
Date 2015-03-03 end: 2613.637146949768
Date 2015-03-04 end: 2618.361737012863
Date 2015-03-05 end: 2623.0613458156586
Date 2015-03-06 end: 2627.7669620513916
Date 2015-03-07 end: 2632.4401590824127
Date 2015-03-08 end: 2637.113662958145
Date 2015-03-09 end: 2641.821587085724
Date 2015-03-10 end: 2646.8508570194244
Date 2015-03-11 end: 2651.990000963211
Date 2015-03-12 end: 2656.7077028751373
Date 2015-03-13 end: 2661.464975833893
Date 2015-03-14 end: 2666.228704929352
Date 2015-03-15 end: 2670.9254610538483
Date 2015-03-16 end: 2676.0627579689026
Date 2015-03-1

Date 2015-09-16 end: 3592.7997229099274
Date 2015-09-17 end: 3597.757793903351
Date 2015-09-18 end: 3602.578477859497
Date 2015-09-19 end: 3607.606168985367
Date 2015-09-20 end: 3612.687171936035
Date 2015-09-21 end: 3617.525540828705
Date 2015-09-22 end: 3622.485635995865
Date 2015-09-23 end: 3627.395115852356
Date 2015-09-24 end: 3632.2466399669647
Date 2015-09-25 end: 3637.046327829361
Date 2015-09-26 end: 3641.971920967102
Date 2015-09-27 end: 3646.7680747509003
Date 2015-09-28 end: 3651.5085368156433
Date 2015-09-29 end: 3656.277766942978
Date 2015-09-30 end: 3661.0662417411804
Date 2015-10-01 end: 3665.9514520168304
Date 2015-10-02 end: 3670.681975841522
Date 2015-10-03 end: 3675.458561897278
Date 2015-10-04 end: 3680.294357776642
Date 2015-10-05 end: 3685.0686419010162
Date 2015-10-06 end: 3689.9460797309875
Date 2015-10-07 end: 3694.8914210796356
Date 2015-10-08 end: 3699.7345707416534
Date 2015-10-09 end: 3704.431599855423
Date 2015-10-10 end: 3709.181485891342
Date 2015-10-11

Date 2015-04-13 end: 4519.603903055191
Date 2015-04-14 end: 4523.28019785881
Date 2015-04-15 end: 4526.973313808441
Date 2015-04-16 end: 4530.706025123596
Date 2015-04-17 end: 4534.474083900452
Date 2015-04-18 end: 4538.171351909637
Date 2015-04-19 end: 4541.856376886368
Date 2015-04-20 end: 4545.622781038284
Date 2015-04-21 end: 4549.520359039307
Date 2015-04-22 end: 4553.230080842972
Date 2015-04-23 end: 4557.113924980164
Date 2015-04-24 end: 4560.780200004578
Date 2015-04-25 end: 4564.441260814667
Date 2015-04-26 end: 4568.13063788414
Date 2015-04-27 end: 4571.831040143967
Date 2015-04-28 end: 4575.51407289505
Date 2015-04-29 end: 4579.291263103485
Date 2015-04-30 end: 4583.022380828857
Date 2015-05-01 end: 4586.919294118881
Date 2015-05-02 end: 4590.746017932892
Date 2015-05-03 end: 4594.415018081665
Date 2015-05-04 end: 4598.178837060928
Date 2015-05-05 end: 4601.844908952713
Date 2015-05-06 end: 4605.584208011627
Date 2015-05-07 end: 4609.3283360004425
Date 2015-05-08 end: 4612.9

Date 2015-11-10 end: 5302.885857820511
Date 2015-11-11 end: 5306.531074762344
Date 2015-11-12 end: 5310.187274694443
Date 2015-11-13 end: 5313.816988945007
Date 2015-11-14 end: 5317.457236051559
Date 2015-11-15 end: 5321.1026220321655
Date 2015-11-16 end: 5324.745388746262
Date 2015-11-17 end: 5328.447777748108
Date 2015-11-18 end: 5332.132919788361
Date 2015-11-19 end: 5335.77552485466
Date 2015-11-20 end: 5339.4266567230225
Date 2015-11-21 end: 5343.070456981659
Date 2015-11-22 end: 5346.708883047104
Date 2015-11-23 end: 5350.347023010254
Date 2015-11-24 end: 5353.97661280632
Date 2015-11-25 end: 5357.651387929916
Date 2015-11-26 end: 5361.298512935638
Date 2015-11-27 end: 5364.945822954178
Date 2015-11-28 end: 5368.619260072708
Date 2015-11-29 end: 5372.270720720291
Date 2015-11-30 end: 5375.914509773254
Date 2015-12-01 end: 5379.569034814835
Date 2015-12-02 end: 5383.225468873978
Date 2015-12-03 end: 5387.101288080215
Date 2015-12-04 end: 5390.733631134033
Date 2015-12-05 end: 5394

In [14]:
df_db

,holiday,date,time,line,order,from_stt,to_stt,in,out,cong,crowd
0,2,2015-01-01,5,1호선,상선,서울역,시청,441.0,392.0,NaN,NaN
1,2,2015-01-01,5,1호선,상선,시청,종각,101.0,57.0,NaN,NaN
2,2,2015-01-01,5,1호선,상선,종각,종로3가,898.0,60.0,NaN,NaN
3,2,2015-01-01,5,1호선,상선,종로3가,종로5가,371.0,27.0,NaN,NaN
4,2,2015-01-01,5,1호선,상선,종로5가,동대문,57.0,22.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1605995,0,2015-12-31,24,4호선,하선,쌍문,수유,197.0,1128.0,NaN,NaN
1605996,0,2015-12-31,24,4호선,하선,창동,쌍문,200.0,987.0,NaN,NaN
1605997,0,2015-12-31,24,4호선,하선,노원,창동,346.0,1104.0,NaN,NaN
1605998,0,2015-12-31,24,4호선,하선,상계,노원,116.0,717.0,NaN,NaN


In [16]:
df_db.to_csv(f"./final_db/df_db_hday_rvs_inout.csv")